In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('../rtl-spectrogram/pll'))

from Orig_PLL import PhaseLockedLoop
from Fast_PLL import Fast_PhaseLockedLoop

import numpy as np

K = 1
duration_secs = 2 / K

for fs in [150 * K * 10, 150 * K * 100, 200 * K * 10]:
    in_f_Hz   = 10*K
    fs = 150 * K * 10

    t = np.linspace(0,duration_secs, int(duration_secs*fs))
    in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

    for K_p in [0.2667/10, 0.2667, 0.1337/10,]:
        for K_i in [0.0178*100, 0.0178, 0.0178*10, 0.1337]:
            for K_0 in [1, 2, 5]:
                for in_f_Hz in [ 9900 ]:
                    pll = PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )
                    fast_pll = Fast_PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )

                    for idx in range(len(in_sig)):
                        orig_cosOut, orig_sinOut, orig_e_D, orig_e_F = pll.proc( in_sig[idx] )

                        fast_cosOut, fast_sinOut, fast_e_D, fast_e_F = fast_pll.proc( in_sig[idx] )

                        assert( orig_cosOut == fast_cosOut )
                        assert( orig_sinOut == fast_sinOut )
                        assert( orig_e_D == fast_e_D )
                        assert( orig_e_F == fast_e_F )
                    
                    

print( "All passed!" )

All passed!


In [3]:
import time

K = 1
duration_secs = 10 / K

start = time.time()
for fs in [256000]:
    in_f_Hz   = 10*K

    t = np.linspace(0,duration_secs, int(duration_secs*fs))
    in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

    for K_p in [0.2667/10]:
        for K_i in [0.0178*100]:
            for K_0 in [1]:
                for in_f_Hz in [ 9900 ]:
                    pll = PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )

                    for idx in range(len(in_sig)):
                        orig_cosOut, orig_sinOut, orig_e_D, orig_e_F = pll.proc( in_sig[idx] )

end = time.time()
print( "original:", end-start )

start = time.time()
for fs in [256000]:
    in_f_Hz   = 10*K

    t = np.linspace(0,duration_secs, int(duration_secs*fs))
    in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

    for K_p in [0.2667/10]:
        for K_i in [0.0178*100]:
            for K_0 in [1]:
                for in_f_Hz in [ 9900 ]:
                    fast_pll = Fast_PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )

                    for idx in range(len(in_sig)):
                        fast_cosOut, fast_sinOut, fast_e_D, fast_e_F = fast_pll.proc( in_sig[idx] )
end = time.time()
print( "fast:", end-start )

original: 12.26938271522522
fast: 11.091150283813477
